# Challenge 3

In this challenge we will work on the `Orders` data set. In your work you will apply the thinking process and workflow we showed you in Challenge 2.

You are serving as a Business Intelligence Analyst at the headquarter of an international fashion goods chain store. Your boss today asked you to do two things for her:

**First, identify two groups of customers from the data set.** The first group is **VIP Customers** whose **aggregated expenses** at your global chain stores are **above the 95th percentile** (aka. 0.95 quantile). The second group is **Preferred Customers** whose **aggregated expenses** are **between the 75th and 95th percentile**.

**Second, identify which country has the most of your VIP customers, and which country has the most of your VIP+Preferred Customers combined.**

## Q1: How to identify VIP & Preferred Customers?

We start by importing all the required libraries:

In [1]:
# import required libraries
import numpy as np
import pandas as pd

Next, extract and import `Orders` dataset into a dataframe variable called `orders`. Print the head of `orders` to overview the data:

In [11]:
import zipfile
orders_zip = zipfile.ZipFile('Orders.zip', mode = 'r')
orders_zip.printdir()

File Name                                             Modified             Size
Orders.csv                                     2018-10-31 07:03:36     46169568


In [48]:
# I could not load the file directly from the zip. I tried different methods and all ended up destroying the file.
#orders = orders_zip.read('Orders.csv')
#orders.head()

In [2]:
orders = pd.read_csv('Orders.csv')
orders.head()

,Unnamed: 0,InvoiceNo,StockCode,year,month,day,hour,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,amount_spent
0,0,536365,85123A,2010,12,3,8,white hanging heart t-light holder,6,2010-12-01 08:26:00,2.55,17850,United Kingdom,15.30
1,1,536365,71053,2010,12,3,8,white metal lantern,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34
2,2,536365,84406B,2010,12,3,8,cream cupid hearts coat hanger,8,2010-12-01 08:26:00,2.75,17850,United Kingdom,22.00
3,3,536365,84029G,2010,12,3,8,knitted union flag hot water bottle,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34
4,4,536365,84029E,2010,12,3,8,red woolly hottie white heart.,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34


In [3]:
def standardize_columns(df):
    '''
This functions replaces the names of the columns by the lower case version,
blank spaces for underscores '_' and drops the column "Unnamed: 0" with index values
input: data frame
    '''
    df.columns = df.columns.str.lower().str.replace(' ', '_')
    return df.drop(['unnamed:_0'], axis=1, inplace = True)

standardize_columns(orders)
orders.head()

,invoiceno,stockcode,year,month,day,hour,description,quantity,invoicedate,unitprice,customerid,country,amount_spent
0,536365,85123A,2010,12,3,8,white hanging heart t-light holder,6,2010-12-01 08:26:00,2.55,17850,United Kingdom,15.30
1,536365,71053,2010,12,3,8,white metal lantern,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34
2,536365,84406B,2010,12,3,8,cream cupid hearts coat hanger,8,2010-12-01 08:26:00,2.75,17850,United Kingdom,22.00
3,536365,84029G,2010,12,3,8,knitted union flag hot water bottle,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34
4,536365,84029E,2010,12,3,8,red woolly hottie white heart.,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34


---

"Identify VIP and Preferred Customers" is the non-technical goal of your boss. You need to translate that goal into technical languages that data analysts use:

## How to label customers whose aggregated `amount_spent` is in a given quantile range?


We break down the main problem into several sub problems:

#### Sub Problem 1: How to aggregate the  `amount_spent` for unique customers?

#### Sub Problem 2: How to select customers whose aggregated `amount_spent` is in a given quantile range?

#### Sub Problem 3: How to label selected customers as "VIP" or "Preferred"?

*Note: If you want to break down the main problem in a different way, please feel free to revise the sub problems above.*

Now in the workspace below, tackle each of the sub problems using the iterative problem solving workflow. Insert cells as necessary to write your codes and explain your steps.

In [4]:
# Subproblem1: aggregate the amount spent by unique customers
#group = orders.groupby(['customerid']).agg({'amount_spent': 'sum'})

group = orders.groupby('customerid')['amount_spent'].sum()
group


customerid
12346    77183.60
12347     4310.00
12348     1797.24
12349     1757.55
12350      334.40
           ...   
18280      180.60
18281       80.82
18282      178.05
18283     2094.88
18287     1837.28
Name: amount_spent, Length: 4339, dtype: float64

Subproblem2: VIP
1. sort values in descending order
2. Select top 5% -> len(group): len*5/100

In [13]:
# version 1
#vip = group.nlargest(int(len(group)*0.05), columns=['amount_spent'])
#display(vip)

,amount_spent,label
customerid,,
14646,280206.02,Other
18102,259657.30,Other
17450,194550.79,Other
16446,168472.50,Other
14911,143825.06,Other
...,...,...
15382,5955.71,Other
14505,5905.64,Other
14415,5900.28,Other


In [18]:
# version 2 -> define quantiles
quantiles = group.quantile([0.75, 0.95])
quantiles

0.75    1661.640
0.95    5840.182
Name: amount_spent, dtype: float64

In [20]:
vip = group[group >= quantiles[0.95]]
vip_df = pd.DataFrame(vip, columns=[ 'amount_spent'])
vip_df

,amount_spent
customerid,
12346,77183.60
12357,6207.67
12359,6372.58
12409,11072.67
12415,124914.53
...,...
18109,8052.97
18139,8438.34
18172,7561.68


In [19]:
preferred = group[(group >= quantiles[0.75]) & (group < quantiles[0.95])]
preferred_df = pd.DataFrame(preferred, columns=['amount_spent'])
preferred_df

,amount_spent
customerid,
12347,4310.00
12348,1797.24
12349,1757.55
12352,2506.04
12356,2811.43
...,...
18259,2338.60
18260,2643.20
18272,3078.58


In [21]:
for l, i in enumerate(orders['customerid']):
    if i in vip_df.index:
        orders.loc[l, 'customer_type'] = "vip"
    elif i in preferred_df.index:
        orders.loc[l, 'customer_type'] = "preferred"
    else:
        orders.loc[l, 'customer_type'] = "other"


In [35]:
orders.head()

,invoiceno,stockcode,year,month,day,hour,description,quantity,invoicedate,unitprice,customerid,country,amount_spent,customer_type
0,536365,85123A,2010,12,3,8,white hanging heart t-light holder,6,2010-12-01 08:26:00,2.55,17850,United Kingdom,15.30,preferred
1,536365,71053,2010,12,3,8,white metal lantern,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34,preferred
2,536365,84406B,2010,12,3,8,cream cupid hearts coat hanger,8,2010-12-01 08:26:00,2.75,17850,United Kingdom,22.00,preferred
3,536365,84029G,2010,12,3,8,knitted union flag hot water bottle,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34,preferred
4,536365,84029E,2010,12,3,8,red woolly hottie white heart.,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34,preferred


In [22]:
orders['customer_type'].value_counts()

preferred    151781
other        141659
vip          104484
Name: customer_type, dtype: int64

Now we'll leave it to you to solve Q2 & Q3, which you can leverage from your solution for Q1:

## Q2: How to identify which country has the most VIP Customers?

In [34]:
orders.columns

Index(['invoiceno', 'stockcode', 'year', 'month', 'day', 'hour', 'description',
       'quantity', 'invoicedate', 'unitprice', 'customerid', 'country',
       'amount_spent', 'customer_type'],
      dtype='object')

In [31]:
orders[orders['customer_type']=='vip']['country']

26                France
27                France
28                France
29                France
30                France
               ...      
397883    United Kingdom
397905    United Kingdom
397906    United Kingdom
397907    United Kingdom
397908    United Kingdom
Name: country, Length: 104484, dtype: object

In [38]:
vip_countries = orders[orders['customer_type'] == 'vip'].groupby('country').agg({'customer_type': 'count'})
vip_country_num1 = vip_countries['customer_type'].idxmax()
print("The country with the most VIP customers is {}.".format(vip_country_num1))

The country with the most VIP customers is United Kingdom.


## Q3: How to identify which country has the most VIP+Preferred Customers combined?

In [45]:
vip_preferred_by_country = orders[orders['customer_type'].isin(['vip', 'preferred'])].groupby(['country', 'customer_type']).size().unstack(fill_value=0)
vip_preferred_by_country.head()

customer_type,preferred,vip
country,,
Australia,130,898
Austria,158,0
Belgium,1503,54
Canada,135,0
Channel Islands,225,364


In [46]:
# add vip and preferred counts together
vip_preferred_by_country['total'] = vip_preferred_by_country['vip'] + vip_preferred_by_country['preferred']
vip_preferred_by_country.head()

customer_type,preferred,vip,total
country,,,
Australia,130,898,1028
Austria,158,0,158
Belgium,1503,54,1557
Canada,135,0,135
Channel Islands,225,364,589


In [47]:
# get the top country with vip-preferred cutomers combined
vip_preferred_country_num1 = vip_preferred_by_country['total'].idxmax()

print("The country with the most VIP and Preferred customers combined is {}.". format(vip_preferred_country_num1))

The country with the most VIP and Preferred customers combined is United Kingdom.
